# Problem Set 1
## Exercise 1

In order to calculate the implied volatility using the Newton method, I created several functions that correspond to different formualae given in the assignement.

I firstly imported the math package that enables the usage of many predefined mathematical functions in Python, for example logarithm function, square root function and exponential function. All of the given values for specific elements  of the option on a stock calculation were written into corresponding variables.

The function *d1()* returns the value of the formula $(2)$, given the input parameter sigma as the implied volatility, and using all of the other variables that have already been defined.

The function *d2()* similarly, returns the value of the formula  $(3)$, given the input parameter sigma as the implied volatility, and calls the function *d1()* in order to calculate the first element of the subtraction.

The function *f()* takes sigma as the input parameter and returns the value obtained using the Black-Sholes option price formula for a (non-dividend-paying) call, which is then reduced by the current value of the option (as given in the formula $(1)$).

The function *df()*, again uses the sigma as the input parameter, and is used for the calculation of the derivative of the option price with respect to volatility (the *vega* of the option), given in the formula $(4)$.

In the end, the *phi()* function is used to calculate the cumulative distribution function for the standard normal distribution for a given input parameter x.

In [1]:
import math

#current value of the stock
S=100
#strike of the option
K=100
#expiration
T=1.5
#risk-free rate
r=0.04
#current value of the option
option_value = 10.78

def d1(sig):
    return (math.log(S/K)+(r+0.5*sig**2)*T)/(sig*math.sqrt(T))

def d2(sig):
    return d1(sig)-sig*math.sqrt(T)

#Black-Sholes option price formula reduced by the current value of the option
def f(sig):
    return S*phi(d1(sig))-K*math.exp(-r*T)*phi(d2(sig)) - option_value

#derivative of the option price with respect to volatility
def df(sig):
    return S*(math.exp(-(d1(sig)**2)/2)/math.sqrt(2*math.pi))*math.sqrt(T)

def phi(x):
    'Cumulative distribution function for the standard normal distribution'
    return (1.0 + math.erf(x / math.sqrt(2.0))) / 2.0

The *newton_method()* function is used to implement the Newton method algorithm that is used for the calculation of the root of the function. It accepts four parameters: the function that is observed, it's first derivative, the starting point and the stopping criterium.

Current distance from the zero of the function is calculated as the absolute value of the function for the given starting point, and the number of iterations is preserved in the variable *i*.

The algorithm keeps on iterating intil the distance becomes lesser than the given stopping criterium. In the meantime, each new point is calculated as the subtraction of the previous point and the fraction of the Black-Scholes function and it's derivative. Afterwards, the updated distance from the root of the function is recalculated and the number of iterations is increased by one.

In the end, the obtained results are printed using the *print* function and string formatting.

In [2]:
def newtons_method(f, df, x0, e):
    #calculate the stopping criterium
    delta = abs(f(x0))
    #number of iterations
    i=0
    #keep on iterating until you fullfil the stopping criterium 
    while delta > e:
        #next point is the previous reduced by the value of fraction of the function and it's derivative
        x0 = x0 - f(x0)/df(x0)
        #recalculate the difference from the stopping criterium
        delta = abs(f(x0))
        #increase the number of iterations
        i+=1
    #print the results
    print ('Number of iterations is: {}'.format(i))
    print ('Root is at: {}'.format(x0))
    print ('f(x) at root is: {}'.format(f(x0)))

Implemented function was tested for the starting point $\sigma = 1$, and the stoping criterium $\epsilon = 1^{-5}$.

In [3]:
newtons_method(f, df, 1, 1e-5)

Number of iterations is: 4
Root is at: 0.1586478996605295
f(x) at root is: 7.85418894366785e-08


The solution was reached in only 4 iterations, and the root is found at approximately $\sigma = 0.159$. The value of the e Black-Scholes option price function reduced by the current value of the option at this point is very close to zero.

## Exercise 2
### The Newton method

In [4]:
import numpy as np

Firstly, I created the funcion *gradientFunc()* that as parameter accepts the array of two values, that represents the point for which the gradient is calculated, and returns the value of the gradient as a numpy matrix.

In [5]:
def gradientFunc(x):
    return np.matrix([[4*(x[0]-2)**3+2*(x[0]-2*x[1])],[-4*(x[0]-2*x[1])]])

The function *hessianFunc()* again accepts the value of a particular point as the input parameter, and returns the matrix with values of the hessian for the given point.

In [6]:
def hessianFunc(x):
    return np.matrix([[12*(x[0]-2)**2+2, -4],[-4, 8]])

The function *newton_method()* accepts two input parameters: the starting point and the tolerance level under which the solution can be accepted. It then calculates the starting value for the second norm of the gradient, using the previously mentioned *gradientFunc()* and the predefined numpy function *numpy.linalg.norm*. The variable *i* will be used to follow the number of iterations required to reach a solution. 

If the starting point is not already within the provided tolerance levels, I will start the calculation of the next point $x_{i+1}$. The previously created functions *hessianFunc()* and *gradientFunc()* are used for the calculation of the Hessian and Gradient for the starting point. The Hessian is then inverted using the predefined numpy function *numpy.linalg.inv*, and then multiplied with the Gradient using the predefined function *numpy.matmul*. Unfortunately, the format of the multiplied matrix is not the same as of the forwarded starting point, so I had to flatten the resulting matrix, transform it into a list and then acces the first element in order to get the vector form. The result is then subtracted from the value of the starting point using a predefined function *numpy.subtract*. Afterwards, the new norm is calculated for the resulting point, and the number of iterations is increased by one.

The same steps will keep on iterating in the while loop until the resulting point is close enough to the required minimum, i.e. until the norm gets within the provided tolerance level.

In [7]:
def newton_m(x0, e):
    #calculate the norm of the gradient
    delta = np.linalg.norm(gradientFunc(x0))
    #number of iterations
    i=0
    #while the norm is larger than tolerance level
    while delta > e:
        #multiply inverted hessian and gradient matrices, then modify them to required vector form
        #and subtract from previous point
        x0 = np.subtract(x0,np.matmul(np.linalg.inv(hessianFunc(x0)), gradientFunc(x0)).flatten().tolist()[0])
        #calculate the new norm
        delta = np.linalg.norm(gradientFunc(x0))
        #increase the number of iterations
        i+=1
    #print the results
    print ('Number of iterations is: {}'.format(i))
    print ('Root is at: {}'.format(x0))

I then tested the created functions for the starting point $x_0=(3,3)$ and the proposed tolerance level of $10^{-6}$.

In [8]:
newton_m([3,3], 10**(-6))

Number of iterations is: 13
Root is at: [2.00513823 1.00256912]


The root was obtained in 13 iterations, and it's value is approximately $x_{13}=(2.005,1.003)$.

The same function was then tested again, just with a different starting point  $x_0=(2,2)$.

In [9]:
newton_m([2,2], 10**(-6))

LinAlgError: Singular matrix

Unfortunately, this time the function could not obtain the result because the inverse of the Hessian function could not be calculated since it required the division by zero (as the determinant of the matrix becomes equal to zero).

### The gradient descent method

The gradient descent method *gradient_desc_m()*, similarly to the Newton method accepts the starting point and the tolerance level as the input parameters, but it adds the step size (learning rate) as an additional parameter that should be forwarded. 

It uses the same previously created function named *gradienFunc()* for the gradient and norm calculation, and the variable *i* will again represent the number of iterations of the algorithm. The difference between the two methods lies in the calculation of the calculation of the next point $x_{i+1}$ in each of the iterations. This time, the obtained Gradient value is multiplied with the value of the learning rate and then subtracted from the value of the previous point. The algorithm keeps iterating until it reaches the required tolernce level.

In [10]:
def gradient_desc_m(x0, s, e):
    #calculate the norm of the gradient
    delta = np.linalg.norm(gradientFunc(x0))
    #number of iterations
    i=0
    #while the norm is larger than tolerance level
    while delta > e:
        #multiply the gradient function with the defined learning rate, modify the format
        #and subtract from previous point
        x0 = np.subtract(x0,(s*gradientFunc(x0)).flatten().tolist()[0])
        #calculate the new norm
        delta = np.linalg.norm(gradientFunc(x0))
        #increase the number of iterations
        i+=1
    #print the results
    print ('Number of iterations is: {}'.format(i))
    print ('Root is at: {}'.format(x0))


The testing of the created method was firsly done for the starting point $x_0=(3,3)$, the proposed tolerance level of $10^{-6}$ and the learning rate of $0.18$.

In [11]:
gradient_desc_m([3,3], 0.18, 10**(-6))

Number of iterations is: 81
Root is at: [2.00000722 1.00000351]


The gradient descent method reached an already seen root $x_{81}=(2, 1)$ but within 81 iteration.

Next the testing of the created method was done for the starting point $x_0=(2,2)$, the tolerance level of $10^{-6}$ and the learning rate of $0.196$.

In [12]:
gradient_desc_m([2,2], 0.196, 10**(-6))

Number of iterations is: 432
Root is at: [1.99999996 1.00000009]


These values of the input parameters helped method reached the root of higher precision than in the previously seen examples, but it required 432 iterations to get to it!

I then tried the same values of input parameters for the method, only varying the given step size to $0.2$.

In [13]:
gradient_desc_m([2,2], 0.2, 10**(-6))

Number of iterations is: 18
Root is at: [nan inf]


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:2: RuntimeWarning: overflow encountered in double_scalars
  
C:\Anaconda\lib\site-packages\ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in subtract
  # Remove the CWD from sys.path while we load stuff.


Unfortunately, the method was unable to reach the root with these parameters, and went to infinity within only 18 iterations. This means that the method diverged because the step size was to large.

In the end, the step size of $0.15$ was tested, without any changes to the other input parameters.

In [14]:
gradient_desc_m([2,2], 0.15, 10**(-6))

Number of iterations is: 24354
Root is at: [2.00653818 1.00326914]


It reached the expected root, but this time the method required a significantly larger number of iterations - 24354. This is due to the fact that the step size was too small, so it required many more steps to finally converge.